In [ ]:
# !pip install sweetviz pycaret[full]

# Imports
---

In [ ]:
import pandas as pd
import numpy as np
# import sweetviz as sv
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Load
---

In [ ]:
df = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv')

df.time = pd.to_datetime(df.time)
test.time = pd.to_datetime(test.time)

df = df.drop('row_id', axis=1)
test = test.drop('row_id', axis=1)

In [ ]:
df.head()

In [ ]:
test.head()

In [ ]:
test.shape

# Analysis
---

In [ ]:
# report = sv.analyze(df, target_feat='congestion')
# report.show_notebook()

# Treatment
---

In [ ]:
df.head()

In [ ]:
# 'EB': [1,0]
# 'NB': [0,1]
# 'SB': [0,-1]
# 'WB': [-1,0]
# 'NE': [1,1]
# 'SW': [-1,-1]
# 'NW': [-1,1]
# 'SE': [1,-1]
    
direction0 = []
direction1 = []
def direction (valor):
    if valor=='EB':
        return direction0.append(1), direction1.append(0)
    elif valor=='NB':
        return direction0.append(0), direction1.append(1)
    elif valor=='SB':
        return direction0.append(0), direction1.append(-1)
    elif valor=='WB':
        return direction0.append(-1), direction1.append(0)
    elif valor=='NE':
        return direction0.append(1), direction1.append(1)
    elif valor=='SW':
        return direction0.append(-1), direction1.append(-1)
    elif valor=='NW':
        return direction0.append(-1), direction1.append(1)
    elif valor=='SE':
        return direction0.append(1), direction1.append(-1)
    
    
df.direction.map(direction)

direction_s0 = pd.Series(direction0, name='direction0')
direction_s1 = pd.Series(direction1, name='direction1')

df = pd.concat([df, direction_s0, direction_s1], axis=1, names=['direction0', 'direction1'], ignore_index=False)

df = df.drop('direction', axis=1)
df

In [ ]:
minute = []
hour = []
day = []
day_of_week = []

def timeForMinute (valor):
    minute.append(valor.minute)
    
def timeForHour (valor):
    hour.append(valor.hour)
    
def timeForDay (valor):
    day.append(valor.day)

def timeForDayOfWeek (valor):
    day_of_week.append(valor.day_of_week)
    
def timeConvert (valor):
    timeForMinute(valor)
    timeForHour(valor)
    timeForDay(valor)
    timeForDayOfWeek(valor)
    
df.time.map(timeConvert)

In [ ]:
minute_s = pd.Series(minute, name='minute')
hour_s = pd.Series(hour, name='hour')
day_s = pd.Series(day, name='day')
day_of_week_s = pd.Series(day_of_week, name='day_of_week')

In [ ]:
df = pd.concat([df, minute_s, hour_s, day_s, day_of_week_s],
                axis=1,
                names=['minute', 'hour', 'day', 'day_of_week_s'],
                ignore_index=False)
df = df.drop('time', axis=1)
df

In [ ]:
df['xy'] = df['x']+df['y']
df

In [ ]:
# test treatment

# reset arrays
direction0 = []
direction1 = []

minute = []
hour = []
day = []
day_of_week = []

# direction
test.direction.map(direction)

direction_s0 = pd.Series(direction0, name='direction0')
direction_s1 = pd.Series(direction1, name='direction1')

test = pd.concat([test, direction_s0, direction_s1], axis=1, names=['direction0', 'direction1'], ignore_index=False)

test = test.drop('direction', axis=1)

# time
test.time.map(timeConvert)

minute_s = pd.Series(minute, name='minute')
hour_s = pd.Series(hour, name='hour')
day_s = pd.Series(day, name='day')
day_of_week_s = pd.Series(day_of_week, name='day_of_week')

test = pd.concat([test, minute_s, hour_s, day_s, day_of_week_s],
                axis=1,
                names=['minute', 'hour', 'day', 'day_of_week_s'],
                ignore_index=False)
test = test.drop('time', axis=1)

# xy
test['xy'] = test['x']+test['y']

# final
test_final = test
test_final

# Performance Metrics
---

In [ ]:
def mae (y_true, y_pred):
    pred_rounded = y_pred.round().astype(int)
    return mean_absolute_error(y_true, pred_rounded)

# Training
---

In [ ]:
X = df.drop(['congestion'], axis=1)
y = df.congestion

X.shape, y.shape

In [ ]:
X.info()

In [ ]:
x_train = X[:590001]
x_test = X[590001:]

y_train = y[:590001]
y_test = y[590001:]

In [ ]:
model_cb = CatBoostRegressor(verbose=False)

model_cb.fit(x_train, y_train)

y_pred = model_cb.predict(x_test)
y_pred_rounded = y_pred.round().astype(int)
print('MAE:', mae(y_test, y_pred_rounded))

In [ ]:
# 8.9969
# 8.9755
# 8.9679
# 8.9969
# 8.9755
# 7.6285
# 6.83
# 6.7988 - ultimo tratamento dos dados

# Predict
---

In [ ]:
model_cb.fit(X,y)

pred = model_cb.predict(test_final)
pred_rounded = pred.round().astype(int)

# Submission
---

In [ ]:
for i in pred:
    if i<0 or i>100:
        print('Valor incorreto encontrado')

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-mar-2022/sample_submission.csv')
sub['congestion'] = pred_rounded
sub.to_csv('submission.csv', index=False)
sub